# 3.0 - Final Model Training (HuggingFace)
This is your Hugging Face training loop but step-by-step.

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# --- Load dataset ---
df = pd.read_csv("data/processed/combined_dataset.csv")

X = df["text"].astype(str).tolist()
y = df["label"].astype(int).tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [ ]:
# --- Tokenization ---
model_name = "xlm-roberta-base"   # can replace with config
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

train_dataset = Dataset.from_dict({**train_encodings, "labels": train_labels})
val_dataset = Dataset.from_dict({**val_encodings, "labels": val_labels})


In [ ]:
# --- Define model ---
num_labels = len(set(y))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


In [ ]:
# --- Training arguments ---
training_args = TrainingArguments(
    output_dir="models/checkpoints/",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=50,
    no_cuda=True,    # set False if GPU is available
    fp16=False
)


In [ ]:
# --- Define metrics ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

In [ ]:
# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
# --- Save final model ---
trainer.save_model("models/final/")
tokenizer.save_pretrained("models/final/")
